In [20]:
import os
import datetime
import tensorflow as tf
from tensorflow import keras
from keras.layers import(Flatten,Dense,Reshape,Conv2D,MaxPool2D,Conv2DTranspose,concatenate)
from keras.models import Sequential
import matplotlib as plt
import cv2
import numpy as np

In [18]:
def Dataloader() :
    images =[]
    masks =[]

    path = "/home/carl/Downloads/detection/datasets/trainset_March_10"

    file = os.listdir(path)
    print("file=",file)
    for folder_name in os.listdir(path):
        folder_path = os.path.join(path,folder_name)
        if os.path.isdir(folder_path):
            for filename in os.listdir(folder_path):
                if "_labelids_1024x512.png" in filename:
                    mask_path = os.path.join(folder_path,filename)
                elif "image_1024x512.png" in filename:
                    image_path = os.path.join(folder_path,filename)
                
                if os.path.exists(image_path):
                    image = cv2.imread(image_path)
                
                if os.path.exists(mask_path):
                    mask = cv2.imread(mask_path,cv2.IMREAD_GRAYSCALE)
                
                images.append(image)
                masks.append(mask)
    images = np.array(images)
    masks = np.array(masks)

    return images,masks


                            

        


In [14]:
images,masks = Dataloader()

file= ['raw_images']


In [ ]:
# Preprocessing
images = images.astype("float32")/255
masks = masks.astype("float32")/255


In [ ]:
images.shape
masks.shape

In [23]:
def unet_model(input_shape):
   
    model = Sequential()

    #Encoder
    model.add(Conv2D(64,3,activation="relu",padding="same",input_shape=input_shape))
    model.add(Conv2D(64,3,activation="relu",padding="same"))
    model.add(MaxPool2D(pool_size=(2,2)))

    model.add(Conv2D(128,3,activation="relu",padding="same"))
    model.add(Conv2D(128,3,activation="relu",padding="same"))
    model.add(MaxPool2D(pool_size=(2,2)))

    model.add(Conv2D(256,3,activation="relu",padding="same"))
    model.add(Conv2D(256,3,activation="relu",padding="same"))
    model.add(MaxPool2D(pool_size=(2,2)))

    #Middle

    model.add(Conv2D(512,3,activation="relu",padding="same"))
    model.add(Conv2D(512,3,activation="relu",padding="same"))
    
    #Decoder

    model.add(Conv2DTranspose(256,(2,2),strides=(2,2),padding="same"))
    
    model.add(Conv2D(256,3,activation="relu",padding="same"))
    model.add(Conv2D(256,3,activation="relu",padding="same"))
    model.add(concatenate(axis=3))


    model.add(Conv2DTranspose(128,(2,2),strides=(2,2),padding="same"))
    model.add(Conv2D(128,3,activation="relu",padding="same"))
    model.add(Conv2D(128,3,activation="relu",padding="same"))
    model.add(concatenate(axis=3))


    model.add(Conv2DTranspose(64,(2,2),strides=(2,2),padding="same"))
    model.add(Conv2D(64,3,activation="relu",padding="same"))
    model.add(Conv2D(64,3,activation="relu",padding="same"))
    model.add(concatenate(axis=3))

    #output

    model.add(Conv2D(1,1,activation="sigmoid"))

    model.compile(optimizer="adam",loss="binary_crossentropy",metrics=['accuracy'])

    return model

    







In [24]:
input_shape = (256,256,3)
model = unet_model(input_shape)
model.summary

TypeError: concatenate() missing 1 required positional argument: 'inputs'